In [69]:
"""
Загрузка данных
"""
import re
import os
import pandas as pd
import pyodbc
import datetime
import xml.etree.ElementTree as ET
import xmljson
from datetime import datetime

pd.set_option('max_colwidth', 160)
pd.set_option('mode.chained_assignment', None)

conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"\
    r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"


def new_transactions(syn_acc, anal_acc, transactions, beg='2020-04-01'):
    """Подготовка таблицы для загрузки."""

    conn = pyodbc.connect(conn_str)
    if syn_acc == 661 and anal_acc == 1:
        SQL = "SELECT [003_Факты].Дата,[003_Факты].СобНаим, [004_Проводки].Оборот \
                FROM 003_Факты INNER JOIN 004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
                WHERE ([004_Проводки].СинтСчётНом={} AND [004_Проводки].АналСчётНом={}) \
                OR ([004_Проводки].СинтСчётНом=661 AND [004_Проводки].АналСчётНом=3)"\
                .format(syn_acc, anal_acc)
    else:
        SQL = "SELECT [003_Факты].Дата,[003_Факты].СобНаим, [004_Проводки].Оборот \
                FROM 003_Факты INNER JOIN 004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
                WHERE [004_Проводки].СинтСчётНом={} AND [004_Проводки].АналСчётНом={}"\
                .format(syn_acc, anal_acc)
    base = pd.read_sql(SQL, conn)
    df = pd.merge(transactions, base, how='left',
                  left_on=['date', 'amount'],
                  right_on=['Дата', 'Оборот'])
    new_transactions = df[df['Оборот'].isna()].iloc[:, :3]
    new_transactions = new_transactions[new_transactions['date']
                                        >= beg]
    conn.close()

    return new_transactions


def upload_to_base(syn_acc, anal_acc, transactions, beg='2020-04-01'):
    """Функция загрузки новых проводок в базу."""

    df = new_transactions(syn_acc, anal_acc, transactions, beg)

    # Определение типа операций
    conn = pyodbc.connect(conn_str)
    SQL = 'SELECT ID, Description, Name, ds, da, cs, ca \
        FROM 008_LEARNING \
        WHERE ds={} AND da={}'.format(syn_acc, anal_acc)
    base = pd.read_sql(SQL, conn)

    # Проведение прододок
    counter = 0
    for i, base_row in base.iterrows():
        lookup = base_row['Description']
        post = base_row['Name']
        df1 = df[df['description'].str.contains(lookup, case=False) == True]
        for index, row in df1.iterrows():
            with conn.cursor() as crsr:
                crsr.execute("INSERT INTO 003_Факты ( Дата, СобНаим ) VALUES(?, ?)",
                             row['date'], post)
                post_id = crsr.execute("SELECT Max(ФактНом) FROM 003_Факты WHERE СобНаим = ?",
                                       post).fetchall()[0][0]
                crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, 0)'.
                             format(post_id, 0, syn_acc, anal_acc, row['amount']))
                crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, 0)'.
                             format(post_id, 1, base_row['cs'], base_row['ca'], -row['amount']))
                counter += 1
    conn.close()
    print('Счёт {}-{} - добавлено документов: {}'.format(syn_acc, anal_acc, counter))


def vtb_bank_details(path):
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0
    for file in files:
        data = pd.read_csv(os.path.join(directory, path, file),
                           sep=';', encoding='cp1251', header=6)
        if counter == 0:
            transactions = data.loc[data['Статус'] == 'Исполнено']
            counter += 1
        else:
            transactions = transactions.append(data.loc[data['Статус'] == 'Исполнено'],
                                               ignore_index=True)
    transactions = transactions.drop_duplicates()
    transactions = transactions.iloc[:, [1, 5, 7]]
    transactions.columns = ['date', 'amount', 'description']
    transactions['date'] = pd.to_datetime(pd.to_datetime(transactions['date'],
                                                         format='%Y-%m-%d').dt.date)
    transactions['amount'] = [float(x.replace(',', '.').replace(' ', ''))
                              for x in transactions['amount']]
    transactions['description'] = transactions['description'].fillna('Пропуск')

    return transactions


def sberbank_details(path):
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Сбербанк/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0
    for file in files:
        data = pd.read_html(os.path.join(directory, path, file))[6]
        if counter == 0:
            transactions = data.loc[data['ДАТА'] != 'Вчера']
            counter += 1
        else:
            transactions = transactions.append(data.loc[data['ДАТА'] != 'Вчера'],
                                               ignore_index=True)
    transactions = transactions.drop_duplicates()
    transactions = transactions[transactions['ОПЕРАЦИЯ'].str.contains(
        'Перевод между своими счетами и картами') != True]
    transactions = transactions[transactions['ОПЕРАЦИЯ'].str.contains(
        'Показать по') != True]
    transactions = transactions[transactions['ОПЕРАЦИЯ'].str.contains(
        'сполнен') == True]
    transactions = transactions.dropna(subset=['ПОЛУЧАТЕЛЬ'])
    transactions = transactions.dropna(subset=['СУММА']).iloc[:, :-2]
    transactions['ОПЕРАЦИЯ'] = transactions['ОПЕРАЦИЯ'].map(lambda x: x[:-9])
    transactions['СЧЕТ СПИСАНИЯ'] = transactions['СЧЕТ СПИСАНИЯ'].map(
        lambda x: str(x)[-5:])
    transactions['description'] = transactions['ПОЛУЧАТЕЛЬ'] + ' ' +\
        transactions['ОПЕРАЦИЯ']
    transactions.description = transactions.description.apply(
        lambda x: x.replace('•••• ', '').replace('.', ''))
    transactions['date'] = pd.to_datetime(
        transactions['ДАТА']+str(datetime.now().year), format='%d.%m%Y')
    transactions['СУММА'] = transactions['СУММА'].map(lambda x: str(x)[:-5])
    transactions['СУММА'] = [float(x.replace(',', '.').replace(' ', ''))
                             for x in transactions['СУММА']]
    transactions['amount'] = transactions['СУММА']
    return transactions.iloc[:, -3:].sort_values(by=['date'])


def ip_vtb_bank_details(path):
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Предпринимательская деятельность/Договор банковского счёта\/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0
    for file in files:
        data = pd.read_csv(os.path.join(directory, path, file),
                           sep=';', encoding='cp1251', header=0)
        if counter == 0:
            transactions = data.iloc[2:]
            counter += 1
        else:
            transactions = transactions.append(data.iloc[2:],
                                               ignore_index=True)
    transactions = transactions.drop_duplicates()
    transactions['Основание платежа'] = transactions['Счет Получателя'] + \
        ' '+transactions['Основание платежа']
    transactions = transactions.iloc[:, [1, 4, 6]]
    transactions.columns = ['date', 'amount', 'description']
    transactions['date'] = pd.to_datetime(pd.to_datetime(transactions['date'],
                                                         format='%d.%m.%Y').dt.date)
    transactions['amount'] = [float(x.replace(',', '.').replace(' ', ''))
                              for x in transactions['amount']]
    transactions['description'] = transactions['description'].fillna('Пропуск')

    return transactions


def read_vtb_broker_reports(path):
    """Считываем данные по движению денежных средств из брокерского отчёта Банка ВТБ в формате xml"""

    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0

    # Считываем данные из файлов
    for file in files:
        tree = ET.parse(os.path.join(directory, path, file))
        root = tree.getroot()
        for i in range(2):
            json = xmljson.gdata.data(root[11][0][i][0])
            json = json['{report577p_v1}Подробности16_Collection']['{report577p_v1}Подробности16']
            data = pd.DataFrame(json)
            if counter == 0:
                transactions = data
                counter += 1
            else:
                transactions = transactions.append(data, ignore_index=True)

    # Обработка данных
    transactions.columns = ['date', 'amount',
                            'currency', 'description', 'notes']
    transactions = transactions[~transactions.description.isin(
        ['Списание денежных средств', 'Зачисление денежных средств'])]
    transactions = transactions[transactions.currency == 'RUR']
    transactions.date = pd.to_datetime(transactions.date, format='%Y-%m-%d')
    transactions.description = transactions.description + \
        '; ' + transactions.notes.fillna('')
#     transactions = transactions[transactions.description !=
#                                 'Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹']
    transactions = transactions[['date', 'amount', 'description']]
    transactions = transactions.sort_values(by=['date'])

    return transactions


def vtb_broker_security_deals(path):
    """Считываем данные по движению денежных средств из брокерского отчёта Банка ВТБ в формате xml"""

    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
    files = os.listdir(os.path.join(directory, path))
    files = [os.path.join(directory, path, file) for file in files]
    file = max(files, key=os.path.getctime)

    # Считываем данные из файлов
    tree = ET.parse(file)
    root = tree.getroot()
    json = xmljson.gdata.data(root[17][0])
    json = json['{report577p_v1}Подробности9_Collection']['{report577p_v1}Подробности9']
    security_deals = pd.DataFrame(json)

    # Обработка данных
    security_deals = security_deals[['NameBeg9', 'curs_datebeg9', 'currency_ISO9',
                                     'NameEnd9', 'currency_paym7', 'deal_cost7',
                                     'bank_сommition7', 'deal_code5', 'deal_code1']]
    security_deals.columns = ['security', 'date', 'deal_type', 'quantity',
                              'amount', 'accumulated_coupon', 'deal_сommission', 'bank_сommission', 'deal_code']
    security_deals.loc[:, 'date'] = pd.to_datetime(
        security_deals.loc[:, 'date'], format='%Y-%m-%d').dt.normalize()
    security_deals.loc[:, 'security'] = security_deals.security.apply(
        lambda x: x[-12:])
    security_deals.loc[:, 'quantity'] = security_deals.apply(
        lambda x: x.quantity if x.deal_type == 'Покупка' else -x.quantity, axis=1)
    security_deals.loc[:, 'amount'] = security_deals.apply(
        lambda x: x.amount if x.deal_type == 'Покупка' else -x.amount, axis=1)
    security_deals.loc[:, 'accumulated_coupon'] = security_deals.apply(
        lambda x: x.accumulated_coupon if x.deal_type == 'Покупка' else -x.accumulated_coupon, axis=1)
    security_deals.loc[:, 'сommission'] = security_deals.deal_сommission + \
        security_deals.bank_сommission
    security_deals = security_deals.iloc[:, [0, 1, 3, 4, 5, 8, 9]]

    return security_deals


def new_security_deals(security_deals):
    """Подготовка таблицы для загрузки."""

    start_date = '2020-01-01'
    conn = pyodbc.connect(conn_str)
    SQL = 'SELECT DISTINCT [003_Факты].transaction_code \
            FROM 003_Факты INNER JOIN \
            004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
            WHERE ((([003_Факты].transaction_code) Is Not Null) AND \
                    ([004_Проводки].СинтСчётНом=581 Or [004_Проводки].СинтСчётНом=582 Or [004_Проводки].СинтСчётНом=662))'
    base = pd.read_sql(SQL, conn)
    security_deals = security_deals[~security_deals.deal_code.isin(
        base['transaction_code'])]
    security_deals = security_deals[security_deals.date >= start_date]

    SQL = 'SELECT СинтСчётНом AS syn_acc, АналСчётНом AS anal_acc, acc_code \
            FROM 002_АналСчета \
            WHERE (СинтСчётНом=581 Or СинтСчётНом=582) AND acc_code Is Not Null'
    mapping = pd.read_sql(SQL, conn)
    security_deals = pd.merge(security_deals, mapping, how='left',
                              left_on=['security'],
                              right_on=['acc_code']).iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8]]
    conn.close()

    return security_deals


def security_deals_upload_to_base(security_deals):
    """Функция загрузки новых проводок в базу."""

    security_deals = new_security_deals(security_deals)
    conn = pyodbc.connect(conn_str)
    cur = conn.cursor()

    # Проведение проводок
    post = 'Купля-продажа ЦБ'
    for i, row in security_deals.iterrows():
        k = 0
        cur.execute('INSERT INTO 003_Факты ( Дата, СобНаим, transaction_code ) VALUES(?, ?, ?)',
                    row['date'], post, row['deal_code'])
        post_id = cur.execute('SELECT Max(ФактНом) FROM 003_Факты WHERE СобНаим = ?',
                              post).fetchall()[0][0]
        cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                    post_id, k, row['syn_acc'], row['anal_acc'],
                    row['amount']-row['accumulated_coupon']+row['сommission'], row['quantity'])
        if row['syn_acc'] == 582:
            k += 1
            cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                        post_id, k, row['syn_acc'], row['anal_acc']+1,
                        row['accumulated_coupon'], None)
        k += 1
        cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                    post_id, k, 760, 53, -row['amount'], None)
        k += 1
        cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                    post_id, k, 760, 57, -row['сommission'], None)
        if row['syn_acc'] == 581:
            k += 1
            cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                        post_id, k, 792, 19, -row['amount']-row['сommission'], None)
            k += 1
            cur.execute('INSERT INTO 004_Проводки VALUES(?, ?, ?, ?, ?, ?)',
                        post_id, k, 792, 20, row['amount']+row['сommission'], None)
    conn.commit()
    conn.close()
    print('Добавлено операций купли-продажи ценных бумаг: {}'.format(len(security_deals)))


def reg_transactions(syn_acc, anal_acc, beg='2020-04-01'):
    conn = pyodbc.connect(conn_str)
    SQL = 'SELECT [003_Факты].Дата,\
            Sum([004_Проводки].Оборот) AS [Sum-Оборот], [004_Проводки].АналСчётНом \
            FROM 003_Факты INNER JOIN 004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
            GROUP BY [004_Проводки].СинтСчётНом, [004_Проводки].АналСчётНом, [003_Факты].Дата \
            HAVING  [004_Проводки].СинтСчётНом={} AND \
                    [004_Проводки].АналСчётНом={} AND \
                    [003_Факты].Дата >={} AND \
                    Sum([004_Проводки].Оборот)<>0'.format(syn_acc, anal_acc, beg)
    base = pd.read_sql(SQL, conn)
    base.columns = ['date', 'amount', 'description']
    base.amount = -base.amount
    base.description = base.description.apply(lambda x: str(x))
    conn.close()

    return base


def upload_data(syn_acc, anal_acc, tr=None, beg='2020-04-01'):
    if type(tr) != type(None):
        transactions = tr
    else:
        transactions = reg_transactions(syn_acc, anal_acc, beg)
    upload_to_base(syn_acc, anal_acc, transactions, beg)
    if len(new_transactions(syn_acc, anal_acc, transactions, beg)) > 0:
        display(new_transactions(syn_acc, anal_acc, transactions, beg))


# Загрузка списка операций по мастер-счёту
path = r'Георгий/Договор комплексного обслуживания/ГМС'
transactions = vtb_bank_details(path)
upload_data(510, 6, tr=transactions)

# Загрузка списка операций по мастер-счёту Алены
path = r'Алена/МС'
transactions = vtb_bank_details(path)
upload_data(510, 7, tr=transactions)

# Загрузка списка операций по накопительному счёту
path = r'Георгий/Договор комплексного обслуживания/Копилка'
transactions = vtb_bank_details(path)
upload_data(510, 12, tr=transactions)

# Загрузка списка операций по накопительному счёту Алены
path = r'Алена/Копилка'
transactions = vtb_bank_details(path)
upload_data(510, 9, tr=transactions)

# Загрузка списка операций по кредитной карте
path = r'Георгий/Договор комплексного обслуживания/КК'
transactions = vtb_bank_details(path)
upload_data(661, 1, tr=transactions)

# Загрузка списка операций по Сбербанку Егора
path = r'Егор/'
transactions = sberbank_details(path)
upload_data(510, 11, tr=transactions)

# Загрузка списка операций предпринимательскому счёту
path = r'Выписки/'
transactions = ip_vtb_bank_details(path)
upload_data(510, 1, tr=transactions)

# Загрузка списка операций по Сбербанку Алены
path = r'Алена/'
transactions = sberbank_details(path)
upload_data(510, 5, tr=transactions)

# Загрузка операций по брокерскому счёту Егора
path = r'Георгий/Брокерские отчёты XML/Основной'
transactions = read_vtb_broker_reports(path)
data = transactions[transactions.description !=
                    'Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹'].copy()
upload_data(761, 0, tr=data, beg='2020-01-01')
data = transactions[transactions.description ==
                    'Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹'].copy()
upload_data(760, 53, tr=data, beg='2020-01-01')
security_deals = vtb_broker_security_deals(path)
security_deals_upload_to_base(security_deals)

# Загрузка операций по ИИС Егора
path = r'Георгий/Брокерские отчёты XML/ИИС'
transactions = read_vtb_broker_reports(path)
data = transactions[transactions.description !=
                    'Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹'].copy()
upload_data(761, 3, tr=data, beg='2020-01-01')
data = transactions[transactions.description ==
                    'Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹'].copy()
upload_data(760, 53, tr=data, beg='2020-01-01')
security_deals = vtb_broker_security_deals(path)
security_deals_upload_to_base(security_deals)

# Загрузка операций по ИИС Алены
path = r'Алена/Брокерские отчёты XML'
transactions = read_vtb_broker_reports(path)
data = transactions[transactions.description !=
                    'Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹'].copy()
upload_data(761, 4, tr=data, beg='2020-01-01')
data = transactions[transactions.description ==
                    'Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹'].copy()
upload_data(760, 53, tr=data, beg='2020-01-01')
security_deals = vtb_broker_security_deals(path)
security_deals_upload_to_base(security_deals)

# Списание расходов
upload_data(760, 31, beg='2020-08-01')  # Профсоюз

Счёт 510-6 - добавлено документов: 0
Счёт 510-7 - добавлено документов: 0
Счёт 510-12 - добавлено документов: 0
Счёт 510-9 - добавлено документов: 0
Счёт 661-1 - добавлено документов: 0
Счёт 510-11 - добавлено документов: 0
Счёт 510-1 - добавлено документов: 0
Счёт 510-5 - добавлено документов: 0
Счёт 761-0 - добавлено документов: 0
Счёт 760-53 - добавлено документов: 0
Добавлено операций купли-продажи ценных бумаг: 0
Счёт 761-3 - добавлено документов: 0
Счёт 760-53 - добавлено документов: 0
Добавлено операций купли-продажи ценных бумаг: 0
Счёт 761-4 - добавлено документов: 0
Счёт 760-53 - добавлено документов: 0
Добавлено операций купли-продажи ценных бумаг: 0
Счёт 760-31 - добавлено документов: 0


In [67]:
conn = pyodbc.connect(conn_str)
syn_acc = 760
anal_acc = 31
SQL =  'SELECT [003_Факты].Дата,\
        Sum([004_Проводки].Оборот) AS [Sum-Оборот], [004_Проводки].АналСчётНом \
        FROM 003_Факты INNER JOIN 004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
        GROUP BY [004_Проводки].СинтСчётНом, [004_Проводки].АналСчётНом, [003_Факты].Дата \
        HAVING  [004_Проводки].СинтСчётНом={} AND \
                [004_Проводки].АналСчётНом={} AND \
                [003_Факты].Дата >=#7/1/2020# AND \
                Sum([004_Проводки].Оборот)<>0'.format(syn_acc, anal_acc)
base = pd.read_sql(SQL, conn)
base.columns = ['date', 'amount', 'description']
conn.close()

t=None
s = base
type(s) == type(None)

False

In [15]:
import pandas as pd
import xlrd
import pyodbc
import datetime
from datetime import datetime, timedelta

def round_date(s):
    return datetime.date(s.year, s.month, s.day)

conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"

# Подготовка датафрейма для загрузки
data = {
    'name': 'Потребительские расходы',
    'date': '20.09.2020',
    'amount': 500,
    'ds': 430,
    'da': 5,
    'dq': 'Null',
    'cs': 501,
    'ca': 2,
    'cq': 'Null'
}

#Загрузка данных
data['date'] = datetime.strptime(data['date'], '%d.%m.%Y')
conn = pyodbc.connect(conn_str)
with conn.cursor() as crsr:
    crsr.execute("INSERT INTO 003_Факты ( Дата, СобНаим ) VALUES(?, ?)", data['date'], data['name'])
    post_id = crsr.execute("SELECT Max(ФактНом) FROM 003_Факты WHERE СобНаим = ?", data['name']).fetchall()[0][0]
    crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, {})'.format(post_id, 0, data['ds'], data['da'],
                                                                                 data['amount'], data['dq']))
    crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, {})'.format(post_id, 1, data['cs'], data['ca'],
                                                                                -data['amount'], data['cq']))
conn.close()

In [8]:
"""Загрузка истории котировок"""

import requests
import json
import pandas as pd
import pyodbc

# Авторизация
user = 'gdokuchaev@yandex.ru'
password = 'jT3-rgT-dRw-Rtg'
auth_url = "https://passport.moex.com/authenticate"
payload = {'user': user, 'password': password}
requests.post(auth_url, data=payload)

# Загрузка списка акций
conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"
conn = pyodbc.connect(conn_str)
shares = list(pd.read_sql('select * from 006_Акции', conn)['SECID'])
conn.close()

#Запрос
date = '2020-09-24'
start = 0
req = {'history_secs':
       'http://iss.moex.com/iss/history/engines/%(engine)s/markets/%(market)s/boards/%(board)s/securities.json?date=%(date)s&start=%(start)s'}
while start % 100 == 0:
    url = req['history_secs'] % {'engine': 'stock',
                                 'market': 'shares',
                                 'board': 'tqbr',
                                 'date': date,
                                 'start': start}
    response = requests.get(url).json()
    temp = pd.DataFrame(response['history']['data'])
    if start == 0:
        df = temp
    else:
        df = df.append(temp)
    start += len(temp)
else:
    df.columns = list(response['history']['metadata'].keys())
    df = df[['TRADEDATE','SHORTNAME','SECID','CLOSE']][df['SECID'].isin(shares)]
    df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
    
# Загрузка данных в Access
conn = pyodbc.connect(conn_str)
with conn.cursor() as crsr:
    crsr.execute("DELETE * FROM 007_Котировки WHERE DATE = ?", pd.to_datetime(date))
for index, row in df.iterrows():
    with conn.cursor() as crsr:
        crsr.execute("INSERT INTO 007_Котировки VALUES(?,?,?)", row["SECID"], row["TRADEDATE"], row["CLOSE"] )
conn.close()

In [10]:
import pandas as pd
import xlrd
import pyodbc
import datetime

conn = pyodbc.connect(conn_str)
SQL = ' SELECT [003_Факты].Дата, [003_Факты].СобНаим, [004_Проводки].ФактНом, [002_АналСчета].СинтСчётНом, [002_АналСчета].АналСчётНом, [002_АналСчета].АналСчётНаим, [004_Проводки].Оборот, [004_Проводки].Кол \
        FROM 003_Факты INNER JOIN (002_АналСчета INNER JOIN 004_Проводки ON ([002_АналСчета].АналСчётНом = [004_Проводки].АналСчётНом) AND ([002_АналСчета].СинтСчётНом = [004_Проводки].СинтСчётНом)) ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
        WHERE ((([003_Факты].Дата)>=#1/1/2020#) AND (([003_Факты].СобНаим)="Купля-продажа ЦБ") AND (([002_АналСчета].СинтСчётНом)=581)) \
        ORDER BY [003_Факты].Дата'
df = pd.read_sql(SQL, conn)

df

DatabaseError: Execution failed on sql ' SELECT [003_Факты].Дата, [003_Факты].СобНаим, [004_Проводки].ФактНом, [002_АналСчета].СинтСчётНом, [002_АналСчета].АналСчётНом, [002_АналСчета].АналСчётНаим, [004_Проводки].Оборот, [004_Проводки].Кол         FROM 003_Факты INNER JOIN (002_АналСчета INNER JOIN 004_Проводки ON ([002_АналСчета].АналСчётНом = [004_Проводки].АналСчётНом) AND ([002_АналСчета].СинтСчётНом = [004_Проводки].СинтСчётНом)) ON [003_Факты].ФактНом = [004_Проводки].ФактНом         WHERE ((([003_Факты].СобНаим)="Купля-продажа ЦБ") AND (([002_АналСчета].СинтСчётНом)=581)) ': ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')

In [118]:
# Комиссия за вычетом РЕПО
round(df[df['СобНаим'].isin(['Депозитарная комиссия','Комиссия за срочные сделки',
                        'Комиссия по сделкам с ЦБ','Проценты по сделкам РЕПО (комиссия)'])]['Оборот'].sum() + 73050.13,2)

0.0

In [122]:
# Сальдо расчётов
round(df[df['СобНаим'].isin([#'Проценты по сделкам РЕПО (разница)',
                             'Расчёты по сделкам с ЦБ'])]['Оборот'].sum(), 2)

-1253592.36

In [121]:
# Зачисление / вывод
round(df[df['СобНаим'].isin(['Вывод с брокерского счёта','Пополнение брокерского счёта',
                        'Корректировка'])]['Оборот'].sum() + 8258316.71 - 9447201.40, 2)

0.0